# Demo 1: Zaawansowane Transformacje PySpark

**Temat:** Advanced PySpark Transformations  
**Czas trwania:** 60 minut  
**Typ:** Live coding demo + teoria

---

## Cele szkoleniowe

- Window Functions: partitionBy, orderBy, rowsBetween, rangeBetween
- Funkcje okienkowe: lag, lead, row_number, dense_rank, rank
- Rolling windows i agregacje ruchome
- Struktury złożone: explode, posexplode, sequence
- JSON processing: from_json, to_json, schema_of_json
- Funkcje datowe: date_trunc, date_add, add_months, last_day

---

## Inicjalizacja środowiska

In [0]:
%run ../00_setup

---

## Część 1: Window Functions - podstawy

### Koncepcja Window Functions

Window Functions pozwalają na wykonywanie obliczeń na "oknach" (partycjach) danych bez aggregacji całego DataFrame.

**Kluczowe elementy:**
- `partitionBy()`: Podział danych na grupy
- `orderBy()`: Sortowanie w ramach partycji
- `rowsBetween()` / `rangeBetween()`: Definicja zakresu okna

**Zastosowania:**
- Ranking (row_number, rank, dense_rank)
- Porównania czasowe (lag, lead)
- Agregacje ruchome (rolling sum, moving average)
- Analiza trendów

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import (
    col, row_number, rank, dense_rank, lag, lead,
    sum as _sum, avg, count, max as _max, min as _min,
    to_date, date_trunc, date_add, add_months, last_day,
    explode, posexplode, sequence, from_json, to_json, schema_of_json,
    current_timestamp, round as _round, lit
)
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, ArrayType
import datetime

### Import bibliotek i funkcji

Importujemy wszystkie potrzebne funkcje PySpark do pracy z:
- **Window Functions**: `Window`, `row_number`, `rank`, `lag`, `lead`
- **Agregacje**: `sum`, `avg`, `count`, `max`, `min`
- **Funkcje datowe**: `to_date`, `date_trunc`, `date_add`
- **Struktury złożone**: `explode`, `posexplode`, `sequence`
- **JSON**: `from_json`, `to_json`, `schema_of_json`
- **Typy danych**: `StructType`, `StructField`, etc.

In [ ]:
# Przygotowanie przykładowych danych zamówień
orders_data = [
    (1, 1, "2024-01-15", 150.0),
    (2, 2, "2024-01-16", 200.0),
    (3, 1, "2024-02-10", 300.0),
    (4, 3, "2024-02-12", 100.0),
    (5, 2, "2024-03-05", 450.0),
    (6, 1, "2024-03-15", 250.0),
    (7, 3, "2024-03-20", 180.0),
    (8, 2, "2024-04-01", 320.0),
    (9, 1, "2024-04-10", 400.0),
    (10, 3, "2024-04-15", 220.0),
]

### Przygotowanie danych przykładowych

Utworzenie DataFrame z przykładowymi zamówieniami do demonstracji:

### Definicja schematu

Utworzenie schematu dla DataFrame zamówień:

In [ ]:
orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("amount", DoubleType(), False)
])

### Utworzenie DataFrame

Tworzenie DataFrame i konwersja kolumny daty:

In [ ]:
orders_df = spark.createDataFrame(orders_data, orders_schema)
orders_df = orders_df.withColumn("order_date", to_date(col("order_date")))

In [ ]:
orders_df.display()

### Przykład 1: Ranking - row_number, rank, dense_rank

**Różnice:**
- `row_number()`: Unikalne numery (1, 2, 3, 4, ...)
- `rank()`: Z przerwami przy równych wartościach (1, 2, 2, 4, ...)
- `dense_rank()`: Bez przerw przy równych wartościach (1, 2, 2, 3, ...)

In [0]:
# Ranking zamówień dla każdego klienta według kwoty (malejąco)
window_spec = Window.partitionBy("customer_id").orderBy(col("amount").desc())

orders_ranked = orders_df.withColumn("row_num", row_number().over(window_spec)) \
    .withColumn("rank", rank().over(window_spec)) \
    .withColumn("dense_rank", dense_rank().over(window_spec))

In [ ]:
orders_ranked.orderBy("customer_id", "row_num").display()

### Przykład 2: Funkcje lag i lead

**lag()**: Wartość z poprzedniego wiersza  
**lead()**: Wartość z następnego wiersza

**Zastosowania:**
- Porównanie z poprzednim okresem
- Obliczenie zmian (deltas)
- Analiza sekwencji zdarzeń

In [0]:
# Porównanie z poprzednim i następnym zamówieniem
window_spec_time = Window.partitionBy("customer_id").orderBy("order_date")

orders_lag_lead = orders_df \
    .withColumn("prev_amount", lag("amount", 1).over(window_spec_time)) \
    .withColumn("next_amount", lead("amount", 1).over(window_spec_time)) \
    .withColumn("amount_change", col("amount") - col("prev_amount")) \
    .withColumn("amount_change_pct", 
                _round((col("amount") - col("prev_amount")) / col("prev_amount") * 100, 2))

In [ ]:
orders_lag_lead.orderBy("customer_id", "order_date").display()

---

## Część 2: Rolling Windows - agregacje ruchome

### rowsBetween vs rangeBetween

**rowsBetween(start, end)**: Zakres wierszy  
- `Window.unboundedPreceding`: Od początku partycji  
- `Window.unboundedFollowing`: Do końca partycji  
- `-1, 0`: Poprzedni wiersz i bieżący  
- `-2, 0`: Dwa poprzednie + bieżący (3-row window)

**rangeBetween(start, end)**: Zakres wartości (wymaga orderBy)

In [0]:
# Rolling sum - suma krocząca (wszystkie poprzednie + bieżący)
window_cumulative = Window.partitionBy("customer_id") \
    .orderBy("order_date") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

orders_cumulative = orders_df \
    .withColumn("cumulative_sum", _sum("amount").over(window_cumulative)) \
    .withColumn("cumulative_count", count("order_id").over(window_cumulative)) \
    .withColumn("cumulative_avg", _round(avg("amount").over(window_cumulative), 2))

In [ ]:
orders_cumulative.orderBy("customer_id", "order_date").display()

In [0]:
# Moving average - średnia krocząca z 3 ostatnich zamówień
window_moving_3 = Window.partitionBy("customer_id") \
    .orderBy("order_date") \
    .rowsBetween(-2, 0)  # 2 poprzednie + bieżący = 3 wiersze

orders_moving_avg = orders_df \
    .withColumn("moving_avg_3", _round(avg("amount").over(window_moving_3), 2)) \
    .withColumn("moving_sum_3", _sum("amount").over(window_moving_3)) \
    .withColumn("moving_max_3", _max("amount").over(window_moving_3)) \
    .withColumn("moving_min_3", _min("amount").over(window_moving_3))

### Obliczanie średniej kroczącej

Zastosowanie okna ruchomego do obliczenia:
- **moving_avg_3**: Średnia z 3 ostatnich zamówień
- **moving_sum_3**: Suma z 3 ostatnich zamówień  
- **moving_max_3**: Maksimum z 3 ostatnich zamówień
- **moving_min_3**: Minimum z 3 ostatnich zamówień

**rowsBetween(-2, 0)** oznacza: 2 poprzednie wiersze + bieżący = okno 3 wierszy

In [ ]:
orders_moving_avg.orderBy("customer_id", "order_date").display()

---

## Część 3: Struktury złożone - Arrays

### explode i posexplode

**explode()**: Rozwija tablicę do osobnych wierszy  
**posexplode()**: Jak explode, ale z numerem pozycji

**Zastosowania:**
- Normalizacja danych zagnieżdżonych
- Analiza list (tagi, kategorie, produkty)
- Event tracking (sekwencje akcji)

In [0]:
# Przykład: Produkty w zamówieniach (array)
orders_with_products_data = [
    (1, 1, "2024-01-15", ["Laptop", "Mouse", "Keyboard"]),
    (2, 2, "2024-01-16", ["Monitor", "Cable"]),
    (3, 1, "2024-02-10", ["Headphones"]),
    (4, 3, "2024-02-12", ["Tablet", "Case", "Stylus", "Charger"]),
]

**Dane oryginalne (z arrayami)**

Każde zamówienie zawiera listę produktów jako array. Struktura danych:
- `order_id`: ID zamówienia
- `customer_id`: ID klienta
- `order_date`: Data zamówienia
- `products`: Array z nazwami produktów

### Schemat dla danych z produktami

Definicja schematu zawierającego array produktów:

In [ ]:
orders_products_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("products", ArrayType(StringType()), False)
])

### Utworzenie DataFrame z arrayami

Tworzenie DataFrame i wyświetlenie danych oryginalnych:

In [ ]:
orders_products_df.display()

In [ ]:
orders_products_df = spark.createDataFrame(orders_with_products_data, orders_products_schema)

In [0]:
# explode() - rozwija tablicę do osobnych wierszy
orders_exploded = orders_products_df.select(
    "order_id",
    "customer_id",
    "order_date",
    explode("products").alias("product")
)

In [ ]:
orders_exploded.display()

**Wynik:** Array produktów został rozwinięty do osobnych wierszy - każdy produkt ma teraz swój wiersz.

In [0]:
orders_posexploded = orders_products_df.select(
    "order_id",
    "customer_id",
    "order_date",
    posexplode("products").alias("position", "product")
)

In [ ]:
orders_posexploded.display()

**Wynik:** `posexplode()` rozwija array i dodatkowo dodaje kolumnę `position` z numerem pozycji w oryginalnej tablicy (0-indexed).

### Praktyczny przykład: Analiza koszyka zakupowego

**Scenariusz biznesowy:** 
Mamy zamówienia z listą produktów i chcemy przeanalizować:
- Które produkty są najczęściej kupowane razem
- Średnią liczbę produktów w koszyku
- Top produkty per kategoria klienta

In [0]:
# Dane: Zamówienia z produktami, cenami i kategoriami
basket_data = [
    (101, 1, "Premium", "2024-01-15", [
        {"product": "Laptop", "price": 1200.0, "category": "Electronics"},
        {"product": "Mouse", "price": 25.0, "category": "Accessories"},
        {"product": "Keyboard", "price": 75.0, "category": "Accessories"},
        {"product": "USB Cable", "price": 10.0, "category": "Accessories"}
    ]),
    (102, 2, "Standard", "2024-01-16", [
        {"product": "Monitor", "price": 300.0, "category": "Electronics"},
        {"product": "HDMI Cable", "price": 15.0, "category": "Accessories"}
    ]),
    (103, 1, "Premium", "2024-02-10", [
        {"product": "Headphones", "price": 150.0, "category": "Audio"},
        {"product": "Laptop", "price": 1200.0, "category": "Electronics"}
    ]),
    (104, 3, "Standard", "2024-02-12", [
        {"product": "Tablet", "price": 400.0, "category": "Electronics"},
        {"product": "Case", "price": 30.0, "category": "Accessories"},
        {"product": "Stylus", "price": 50.0, "category": "Accessories"},
        {"product": "Charger", "price": 25.0, "category": "Accessories"}
    ]),
    (105, 2, "Premium", "2024-03-05", [
        {"product": "Smartphone", "price": 800.0, "category": "Electronics"},
        {"product": "Screen Protector", "price": 20.0, "category": "Accessories"},
        {"product": "Phone Case", "price": 35.0, "category": "Accessories"}
    ]),
]

In [ ]:
basket_df = spark.createDataFrame(basket_data, basket_schema) \
    .withColumn("order_date", to_date(col("order_date")))

print("Dane oryginalne - zamówienia z zagnieżdżonymi produktami:")
basket_df.display()

In [ ]:
# Schemat dla zagnieżdżonych danych
basket_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("customer_tier", StringType(), False),
    StructField("order_date", StringType(), False),
    StructField("items", ArrayType(
        StructType([
            StructField("product", StringType(), False),
            StructField("price", DoubleType(), False),
            StructField("category", StringType(), False)
        ])
    ), False)
])

**Dane oryginalne - zamówienia z zagnieżdżonymi produktami**

Każde zamówienie zawiera:
- `order_id`: ID zamówienia
- `customer_id`: ID klienta  
- `customer_tier`: Tier klienta (Premium/Standard)
- `order_date`: Data zamówienia
- `items`: Array of structs z produktami (nazwa, cena, kategoria)

In [0]:
# Krok 1: explode() - rozwój zagnieżdżonych produktów
basket_exploded = basket_df.select(
    "order_id",
    "customer_id",
    "customer_tier",
    "order_date",
    explode("items").alias("item")
).select(
    "order_id",
    "customer_id",
    "customer_tier",
    "order_date",
    col("item.product").alias("product"),
    col("item.price").alias("price"),
    col("item.category").alias("category")
)

In [ ]:
basket_exploded.display()

In [ ]:
# Podsumowanie: ile rekordów przed i po explode
orders_before = basket_df.count()
items_after = basket_exploded.count()

print(f"Statystyki:")
print(f"   Zamówień (przed explode): {orders_before}")
print(f"   Pozycji produktów (po explode): {items_after}")

**Wynik:** Dane po explode - każdy produkt w osobnym wierszu

**Statystyki transformacji:**
- Zamówień (przed explode): 5
- Pozycji produktów (po explode): ~15+ (w zależności od liczby produktów w każdym zamówieniu)

In [0]:
# Analiza 1: Top 5 najpopularniejszych produktów
top_products = basket_exploded \
    .groupBy("product", "category") \
    .agg(
        count("*").alias("times_ordered"),
        _sum("price").alias("total_revenue"),
        _round(avg("price"), 2).alias("avg_price")
    ) \
    .orderBy(col("times_ordered").desc()) \
    .limit(5)

In [ ]:
top_products.display()

**TOP 5 Najpopularniejszych produktów**

Analiza pokazuje które produkty są najczęściej zamawiane, wraz z całkowitym przychodem i średnią ceną.

In [ ]:
# Analiza 2: Sprzedaż per kategoria
category_sales = basket_exploded \
    .groupBy("category") \
    .agg(
        count("*").alias("items_sold"),
        _sum("price").alias("revenue"),
        _round(avg("price"), 2).alias("avg_item_price")
    ) \
    .orderBy(col("revenue").desc())

In [ ]:
category_sales.display()

**Sprzedaż per kategoria**

Analiza przychodów według kategorii produktów - pozwala zidentyfikować najważniejsze kategorie biznesowe.

In [0]:
from pyspark.sql.functions import when

# Analiza 3: Metryki koszyka zakupowego per zamówienie
basket_metrics = basket_exploded \
    .groupBy("order_id", "customer_id", "customer_tier", "order_date") \
    .agg(
        count("product").alias("basket_size"),
        _sum("price").alias("order_total"),
        _round(avg("price"), 2).alias("avg_item_price"),
        _max("price").alias("most_expensive_item"),
        count(when(col("category") == "Electronics", 1)).alias("electronics_count"),
        count(when(col("category") == "Accessories", 1)).alias("accessories_count")
    ) \
    .orderBy("order_id")

In [ ]:
basket_metrics.display()

**Metryki koszyka zakupowego**

Analiza każdego zamówienia pod kątem:
- Rozmiar koszyka (liczba produktów)
- Wartość całkowita zamówienia
- Średnia cena produktu w koszyku
- Najdroższy produkt
- Podział na kategorie (Electronics vs Accessories)

In [ ]:
# Podsumowanie per tier klienta
tier_analysis = basket_metrics \
    .groupBy("customer_tier") \
    .agg(
        count("order_id").alias("orders_count"),
        _round(avg("basket_size"), 2).alias("avg_basket_size"),
        _round(avg("order_total"), 2).alias("avg_order_value"),
        _sum("order_total").alias("total_revenue")
    ) \
    .orderBy(col("total_revenue").desc())

In [ ]:
tier_analysis.display()

**Analiza per tier klienta**

Porównanie zachowań zakupowych między segmentami klientów (Premium vs Standard).

In [0]:
# Analiza 4: Market Basket Analysis - produkty kupowane razem

# Przygotowanie: Self-join produktów z tego samego zamówienia
basket_pairs = basket_exploded.alias("a") \
    .join(
        basket_exploded.alias("b"),
        (col("a.order_id") == col("b.order_id")) & 
        (col("a.product") < col("b.product"))  # Unikamy duplikatów (A+B = B+A)
    ) \
    .select(
        col("a.product").alias("product_a"),
        col("b.product").alias("product_b")
    )

# Zlicz najczęstsze pary produktów
product_pairs_count = basket_pairs \
    .groupBy("product_a", "product_b") \
    .agg(count("*").alias("times_together")) \
    .orderBy(col("times_together").desc())

In [ ]:
product_pairs_count.display()

**Produkty kupowane razem (Market Basket Analysis)**

Self-join tego samego zamówienia, aby znaleźć wszystkie pary produktów kupowanych razem. Warunek `product_a < product_b` eliminuje duplikaty.

In [ ]:
# Insight: Które produkty są najczęściej kupowane z Electronics?
electronics_combos = basket_exploded.alias("e") \
    .join(
        basket_exploded.alias("a"),
        (col("e.order_id") == col("a.order_id")) & 
        (col("e.category") == "Electronics") &
        (col("a.category") != "Electronics")
    ) \
    .groupBy(col("e.product").alias("electronics_item"), 
             col("a.product").alias("paired_with")) \
    .agg(count("*").alias("combo_count")) \
    .orderBy(col("combo_count").desc()) \
    .limit(10)

In [ ]:
electronics_combos.display()

**Produkty najczęściej kupowane z Electronics**

Analiza cross-selling: które produkty z innych kategorii są najczęściej kupowane razem z produktami Electronics.

### Kluczowe wnioski z przykładu

**Co zrobiliśmy:**
1. Użyliśmy `explode()` do rozwinięcia zagnieżdżonych produktów (array of structs)
2. Obliczyliśmy metryki per produkt i kategoria
3. Przeanalizowaliśmy koszyki zakupowe (basket size, avg value)
4. Wykonaliśmy Market Basket Analysis (produkty kupowane razem)

**Zastosowania biznesowe:**
- **Rekomendacje produktów**: Produkty często kupowane razem
- **Analiza koszyka**: Średnia wartość, rozmiar per segment klienta
- **Cross-selling**: Które akcesoria sprzedawać z elektroniką
- **Revenue optimization**: Kategorie generujące największy przychód

**Performance tips:**
- `explode()` zwiększa liczbę wierszy - używaj z filterem gdzie możliwe
- Po explode agreguj dane, aby zmniejszyć rozmiar
- Dla bardzo dużych arrayów rozważ partitioning przed explode

### sequence() - generowanie sekwencji

**sequence(start, stop, step)**: Generuje tablicę wartości

**Zastosowania:**
- Generowanie zakresów dat
- Tworzenie series czasowych
- Wypełnianie braków w danych

In [0]:
# Przykład: Generowanie sekwencji dni między datami
from pyspark.sql.functions import expr

date_ranges_data = [
    ("2024-01-01", "2024-01-05"),
    ("2024-02-01", "2024-02-03"),
]

date_ranges_df = spark.createDataFrame(date_ranges_data, ["start_date", "end_date"]) \
    .withColumn("start_date", to_date(col("start_date"))) \
    .withColumn("end_date", to_date(col("end_date")))

In [ ]:
date_sequence.display()

In [ ]:
# Generuj sekwencję dni
date_sequence = date_ranges_df.withColumn(
    "date_array",
    expr("sequence(start_date, end_date, interval 1 day)")
)

**Sekwencja dat jako array**

Funkcja `sequence()` tworzy array dat między `start_date` a `end_date` z krokiem 1 dzień.

In [ ]:
# Rozwój do osobnych wierszy
date_sequence.select(
    "start_date",
    "end_date",
    explode("date_array").alias("date")
).display()

**Sekwencja dat po explode**

Każda data z array zostaje rozwinięta do osobnego wiersza - przydatne do analizy szeregów czasowych.

---

## Część 4: JSON Processing

### from_json, to_json, schema_of_json

**from_json()**: Parsowanie JSON string → struct/array  
**to_json()**: Konwersja struct/array → JSON string  
**schema_of_json()**: Automatyczne wykrycie schematu JSON

**Zastosowania:**
- Parsowanie API responses
- Event tracking (nested JSON events)
- Log processing

In [0]:
# Przykład: Parsowanie JSON payload
json_data = [
    (1, '{"user_id": 101, "action": "click", "metadata": {"page": "home", "duration": 30}}'),
    (2, '{"user_id": 102, "action": "purchase", "metadata": {"page": "checkout", "duration": 120}}'),
    (3, '{"user_id": 101, "action": "view", "metadata": {"page": "product", "duration": 45}}'),
]

In [ ]:
json_df = spark.createDataFrame(json_data, ["event_id", "json_payload"])

**Dane oryginalne (JSON jako string)**

Dane eventów zawierają JSON payload jako string z zagnieżdżoną strukturą:
- `user_id`: ID użytkownika
- `action`: Typ akcji (click, purchase, view)  
- `metadata`: Zagnieżdżone dane (page, duration)

In [0]:
# Automatyczne wykrycie schematu JSON
json_schema = schema_of_json(lit(json_data[0][1]))

# Parsowanie JSON
json_parsed = json_df.withColumn("parsed_data", from_json(col("json_payload"), json_schema))

In [ ]:
json_parsed.display()

**Automatyczne wykrycie schematu i parsowanie JSON**

1. `schema_of_json()` automatycznie wykrywa schemat z przykładowego JSON
2. `from_json()` parsuje string JSON do struct zgodnie z wykrytym schematem

In [0]:
# Wyciąganie pól z zagnieżdżonego JSON
json_flattened = json_parsed.select(
    "event_id",
    col("parsed_data.user_id").alias("user_id"),
    col("parsed_data.action").alias("action"),
    col("parsed_data.metadata.page").alias("page"),
    col("parsed_data.metadata.duration").alias("duration")
)

In [ ]:
json_flattened.display()

**Dane po spłaszczeniu (flattening)**

Wyciąganie konkretnych pól z zagnieżdżonej struktury JSON:
- Dostęp do pól pierwszego poziomu: `parsed_data.user_id`
- Dostęp do zagnieżdżonych pól: `parsed_data.metadata.page`

---

## Część 5: Funkcje datowe i czasowe

### Kluczowe funkcje

**date_trunc()**: Obcięcie do granicy (rok, miesiąc, dzień, godzina)  
**date_add()**: Dodawanie dni  
**add_months()**: Dodawanie miesięcy  
**last_day()**: Ostatni dzień miesiąca  
**datediff()**: Różnica w dniach  
**months_between()**: Różnica w miesiącach

**Zastosowania:**
- Agregacje temporalne (daily, monthly, yearly)
- Analiza cohort
- Retention analysis
- Forecast horizons

In [0]:
from pyspark.sql.functions import datediff, months_between
from pyspark.sql.functions import year, month, dayofweek, quarter

# Przykład: Analiza temporalna zamówień
orders_temporal = orders_df \
    .withColumn("year", year("order_date")) \
    .withColumn("month", month("order_date")) \
    .withColumn("quarter", quarter("order_date")) \
    .withColumn("day_of_week", dayofweek("order_date")) \
    .withColumn("month_start", date_trunc("month", "order_date")) \
    .withColumn("month_end", last_day("order_date")) \
    .withColumn("next_month_start", date_add(last_day("order_date"), 1))

In [ ]:
orders_temporal.display()

### Wynik analizy temporalnej

Dla każdego zamówienia obliczamy:
- **year/month/quarter**: Komponenty czasowe daty zamówienia
- **day_of_week**: Dzień tygodnia (1=niedziela, 7=sobota)
- **month_start**: Pierwszy dzień miesiąca
- **month_end**: Ostatni dzień miesiąca  
- **next_month_start**: Pierwszy dzień następnego miesiąca

**Zastosowania biznesowe:**
- Analiza sezonowości sprzedaży
- Raportowanie per okres (miesięczne, kwartalne)
- Analiza trendów w dniach tygodnia

In [0]:
# Przykład: Obliczanie okresów między zamówieniami
window_date = Window.partitionBy("customer_id").orderBy("order_date")

orders_periods = orders_df \
    .withColumn("prev_order_date", lag("order_date", 1).over(window_date)) \
    .withColumn("days_since_last_order", 
                datediff(col("order_date"), col("prev_order_date"))) \
    .withColumn("months_since_last_order", 
                _round(months_between(col("order_date"), col("prev_order_date")), 2))

In [ ]:
orders_periods.orderBy("customer_id", "order_date").display()

### Analiza okresów między zamówieniami

**Obliczone metryki:**
- **prev_order_date**: Data poprzedniego zamówienia (używając `lag`)
- **days_since_last_order**: Liczba dni od ostatniego zamówienia (`datediff`)
- **months_since_last_order**: Liczba miesięcy od ostatniego zamówienia (`months_between`)

**Zastosowania biznesowe:**
- Analiza częstotliwości zakupów klientów
- Identyfikacja klientów "uśpionych" (długie okresy bez zamówień)
- Segmentacja klientów wg częstotliwości  
- Predykcja następnego zamówienia

---

## Część 6: Praktyczny przykład - Customer Behavior Analysis

### Zadanie: Analiza zachowań klientów

Wykorzystamy wszystkie poznane techniki do kompleksowej analizy:
1. Ranking zamówień dla każdego klienta
2. Porównanie z poprzednim zamówieniem (lag)
3. Średnia ruchoma wydatków
4. Segmentacja temporalna

In [0]:
# Definicja okien dla analizy zachowań klientów
window_customer_time = Window.partitionBy("customer_id").orderBy("order_date")
window_customer_amount = Window.partitionBy("customer_id").orderBy(col("amount").desc())
window_moving_avg = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(-2, 0)
window_cumulative = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Kompleksowa analiza zachowań klientów
customer_behavior = orders_df \
    .withColumn("order_rank", row_number().over(window_customer_amount)) \
    .withColumn("order_sequence", row_number().over(window_customer_time)) \
    .withColumn("prev_amount", lag("amount", 1).over(window_customer_time)) \
    .withColumn("amount_change", col("amount") - col("prev_amount")) \
    .withColumn("moving_avg_3", _round(avg("amount").over(window_moving_avg), 2)) \
    .withColumn("cumulative_spent", _sum("amount").over(window_cumulative)) \
    .withColumn("month", date_trunc("month", "order_date"))

In [ ]:
customer_behavior.orderBy("customer_id", "order_date").display()

---

## Podsumowanie

### Omówione zagadnienia

1. **Window Functions**
   - partitionBy, orderBy
   - row_number, rank, dense_rank
   - lag, lead

2. **Rolling Windows**
   - rowsBetween / rangeBetween
   - Cumulative aggregations
   - Moving averages

3. **Struktury złożone**
   - explode / posexplode
   - sequence()

4. **JSON Processing**
   - from_json, to_json
   - schema_of_json
   - Flattening nested JSON

5. **Funkcje datowe**
   - date_trunc, date_add, add_months, last_day
   - datediff, months_between
   - Temporal aggregations

---

### Best Practices

1. **Window Functions**: Używaj partitionBy dla efektywności
2. **Rolling Windows**: Wybierz odpowiedni zakres (rows vs range)
3. **explode**: Uważaj na performance przy dużych arrayach
4. **JSON**: Wykorzystuj schema_of_json dla automatycznego wykrycia struktury
5. **Temporal**: Standaryzuj strefy czasowe przed analizą

---